<a href="https://colab.research.google.com/github/thanakorn415/ClassWorkAi/blob/main/R1_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

transformers: ใช้โหลด LLM (Language Models)

datasets: ใช้โหลด dataset จาก Hugging Face

sentence-transformers: สำหรับสร้าง vector (embedding) ของข้อความ

faiss-cpu: สำหรับจัดเก็บและค้นหา embedding อย่างรวดเร็ว (Vector Search)

In [ ]:
!pip install -q transformers datasets sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

หลดชุดข้อมูลจาก Hugging Face ที่เกี่ยวกับคำถาม-คำตอบด้านการแพทย์

ใช้เฉพาะ 500 ตัวอย่างแรก (สำหรับทดลอง)

ฟังก์ชัน join_fields จะรวม Question และ Response เป็นข้อความเดียว

ataset.map() ใช้เพื่อแปลงทุกตัวอย่างใน dataset และดึงเฉพาะฟิลด์ "doc" มาเป็น list

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[:500]",
    trust_remote_code=True
)

# สร้างเอกสารจากคำถาม + คำตอบ
def join_fields(example):
    return {
        "doc": f"Q: {example['Question']}\nA: {example['Response']}"
    }

documents = dataset.map(join_fields)["doc"]
documents[:3]  # preview


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['Q: A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\nA: Cystometry in this case of stress urinary incontinence would most likely reveal a normal post-void residual volume, as stress incontinence typically does not involve issues with bladder emptying. Additionally, since stress urinary incontinence is primarily related to physical exertion and not an overactive bladder, you would not expect to see any involuntary detrusor contractions during the test.',
 "Q: A 45-year-old man with a history of alcohol use, who has been abstinent for the past 10 years, presents with sudden onset dysarthria, shuffling gait, and intention tremors. Given this clinical presentation and history, what is the most likely diagnosis?\nA: Considering the clinical p

โหลดโมเดลสำหรับแปลงข้อความเป็น vector (embedding)

encode() แปลงทุกเอกสารเป็นเวกเตอร์ numpy (ใช้ในการค้นหาด้วย FAISS)

สร้าง FAISS Index แบบ L2 (Euclidean distance) สำหรับจัดเก็บ embedding

index.add() ใส่เอกสารทั้งหมดลงในฐานข้อมูล vector เพื่อใช้ค้นหาในภายหลัง

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# โหลด Sentence Transformer สำหรับทำ embedding
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# แปลงทุก document เป็น vector
doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

# สร้าง FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

โหลด tokenizer และ model สำหรับการสร้างข้อความ (LLM)

ใช้โมเดลขนาดเล็ก (1.5B)

pipeline("text-generation", ...) คือ text generator ที่จะใช้ตอบคำถาม

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

rag_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
def rag_ask(query, top_k=3, max_new_tokens=256):
    # แปลง query เป็น embedding
    query_embedding = embedder.encode([query])

    # ค้นหา top_k document ที่ใกล้เคียง
    scores, indices = index.search(np.array(query_embedding), top_k)
    context = "\n\n".join([documents[i] for i in indices[0]])

    # รวม prompt
    prompt = f"""You are a medical expert.
Based on the following context, answer the question.

### Context:
{context}

### Question:
{query}

### Answer:"""

    # Generate คำตอบ
    output = rag_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]["generated_text"]
    return output


In [ ]:
query = "How is hepatitis B transmitted?"
answer = rag_ask(query)
print(answer)


You are a medical expert.
Based on the following context, answer the question.

### Context:
Q: When the genetic material of a virus is isolated and injected into the cytoplasm of a human cell, which specific virus is known to directly produce viable, infectious virions?
A: When the genetic material of a virus is isolated and injected into the cytoplasm of a human cell, the poliovirus is specifically known to directly produce viable, infectious virions. Poliovirus is a positive-sense single-stranded RNA virus, and its RNA can act as mRNA, which the host's ribosomes can immediately translate into viral proteins. This allows the virus to bypass the need for entry into the nucleus, leading to the direct assembly of new virions in the cytoplasm.

Q: A Person suffers from HLA B 27 associated reactive ahritis, urethritis and conjunctivitis. Which is most likely organism involved in this case ?
A. Borrelia burgdorferi
B. Ureaplasma urealyticum
C. Betahemolytic streptococci
D. Streptococcus bo